#Synsthesis data

Notebook này thực hiện nhiệm vụ:
- Tổng hợp ghi chú y tế với mô hình BioGPT đã huấn luyện

In [ ]:
!pip install -q transformers datasets==3.4.1
!pip install sacremoses
!git clone https://github.com/chancholat/mimic.git
!mkdir /content/src
!gdown --folder https://drive.google.com/drive/folders/1b3tBwPYlPx0j-3pdJy4Na6wQBZUTJezv -O /content/src

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.

In [ ]:
import kagglehub
#Download the latest version.
kagglehub.dataset_download('chanhainguyen/thesis-data-process')

100%|██████████| 2.95G/2.95G [00:40<00:00, 78.2MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/chanhainguyen/thesis-data-process/versions/1'

In [ ]:
!mkdir mimic/data
!mv /root/.cache/kagglehub/datasets/chanhainguyen/thesis-data-process/versions/1/processed /content/mimic/data
# !mv /kaggle/input/thesis-data-process/processed /content/mimic/data

## Load Augmented code sets

In [ ]:
import pandas as pd

#read dataframe from csv
df = pd.read_csv('/content/src/augmented_codes_fixed.csv')
augmented_code_sets = [eval(row) for row in df['augmented_code_sets']]
# augmented_code_sets = [codes for codes in matched_rows['augmented codes']]

## Synthesize Notes

In [ ]:
from datasets import load_dataset
from pathlib import Path

dataset_path = Path("mimic/mdace_inpatient_icd9_code.py")
mimic = load_dataset(str(dataset_path), trust_remote_code=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
from datasets import load_dataset

match_note_id = load_dataset("ChanBeDu/augmented_icd9_note_id_full")
# Extract note_ids from dataset_a
note_ids_to_keep = set([ids[0] for ids in match_note_id['train']["match_note_id"] if ids] )
print(len(note_ids_to_keep))
# Filter dataset_b to keep only rows with note_id in dataset_a
real_relevant_dataset = mimic['train'].filter(lambda x: x["note_id"] in note_ids_to_keep)

# Check result
print(len(match_note_id['train']))
print(len(real_relevant_dataset))

2117
3019
2117


### Build common stopping phrase

In [ ]:
import pandas as pd
from collections import Counter
from tqdm import tqdm

def extract_common_phrases(
    notes: list[str],
    lines_from_end: int = 10,
    min_phrase_len: int = 2,
    max_phrase_len: int = 50,   # NEW PARAMETER
    start_char: str = "",
    stop_char: str = "",
    top_k: int = 30
):
    """
    Scan through clinical notes to find commonly occurring ending phrases.

    Args:
        notes: List of clinical note texts.
        lines_from_end: How many lines from the end to extract per note.
        min_phrase_len: Minimum number of words in a phrase to count.
        max_phrase_len: Maximum number of words in a phrase to count.
        top_k: Number of top phrases to return.
    """
    counter = Counter()

    for note in tqdm(notes, desc="Scanning notes"):
        lines = note.strip().splitlines(keepends=True)
        tail_lines = lines[-lines_from_end:]  # Last N lines of the note

        for i, line in enumerate(tail_lines):
            phrase = line.strip(' ')
            word_count = len(phrase.split(' '))
            if min_phrase_len <= word_count <= max_phrase_len and tail_lines[max(0, i-1)].startswith(start_char) and phrase.endswith(stop_char):
                counter[phrase] += 1

    return counter.most_common(top_k)


In [ ]:
# notes = mimic["train"]["text"]
# common_endings = extract_common_stopping_phrases(notes, lines_from_end=10, min_phrase_len=2, max_phrase_len=3, top_k=100)
# stop_phrases = [phrase for phrase, _ in common_endings]
# print()
# for phrase, count in common_endings:
#     print(f"{phrase} — {count} notes")

In [ ]:
discharge_rows = mimic['train'].filter(lambda x: x["note_type"] == "Discharge summary")
notes = discharge_rows["text"]
common_endings = extract_common_phrases(
      notes,
      lines_from_end=15,
      min_phrase_len=2,
      max_phrase_len=5,
      top_k=200
    )
print(len(common_endings))
for phrase, count in common_endings:
    print(f"{phrase.strip()} — {count} notes")

common_endings = [phrase for phrase, _ in common_endings]

Filter:   0%|          | 0/48074 [00:00<?, ? examples/s]

Scanning notes: 100%|██████████| 47904/47904 [00:01<00:00, 26041.01it/s]

200
Followup Instructions: — 24085 notes
Discharge Instructions: — 7554 notes
Discharge Condition: — 5362 notes
Discharge Disposition: — 2825 notes
Discharge Diagnosis: — 2751 notes
Extended Care — 1054 notes
person during off hours** — 963 notes
Dictated By:[**Last Name (NamePattern1) **] — 948 notes
JOB#:  [**Job Number **] — 898 notes
DISCHARGE DIAGNOSES: — 822 notes
Department: CARDIAC SERVICES — 655 notes
Home With Service — 442 notes
Dictated By:[**Last Name (NamePattern1) 344**] — 382 notes
Department: HEMATOLOGY/ONCOLOGY — 335 notes
Follow-Up Appointment Instructions — 328 notes
[**Last Name (NamePattern4) 2138**]p Instructions: — 311 notes
Job#:  [**Job Number **] — 280 notes
Department: [**Hospital3 249**] — 272 notes
DISCHARGE DIAGNOSIS: — 243 notes
Dictated By:[**Dictator Info **] — 210 notes
Please call to schedule appointments — 202 notes
Department: NEUROLOGY — 197 notes
Dictated By:[**Last Name (Titles) **] — 196 notes
Treatments Frequency: — 179 notes
Department: INFEC

In [ ]:
discharge_rows = mimic['train'].filter(lambda x: x["note_type"] == "Discharge summary")
notes = discharge_rows["text"]
common_headings = extract_common_phrases(
      notes,
      lines_from_end=0,
      min_phrase_len=2,
      max_phrase_len=5,
      start_char='\n',
      stop_char=':\n',
      top_k=200
    )
print(len(common_headings))
for phrase, count in common_headings:
    print(f"{phrase.strip()} — {count} notes")

common_headings = [phrase for phrase, _ in common_headings]

Scanning notes: 100%|██████████| 47904/47904 [00:08<00:00, 5495.67it/s]

200
Discharge Disposition: — 35615 notes
Brief Hospital Course: — 33803 notes
Discharge Diagnosis: — 33783 notes
Major Surgical or Invasive Procedure: — 33747 notes
Discharge Condition: — 33743 notes
Past Medical History: — 33668 notes
History of Present Illness: — 33637 notes
Social History: — 33110 notes
Physical Exam: — 33110 notes
Pertinent Results: — 33081 notes
Discharge Instructions: — 33013 notes
Medications on Admission: — 32913 notes
Followup Instructions: — 32842 notes
Family History: — 32721 notes
Discharge Medications: — 32297 notes
DISCHARGE DIAGNOSES: — 6617 notes
PAST MEDICAL HISTORY: — 5003 notes
DISCHARGE MEDICATIONS: — 4886 notes
[**Hospital 93**] MEDICAL CONDITION: — 3118 notes
DISCHARGE DIAGNOSIS: — 2442 notes
MEDICATIONS ON DISCHARGE: — 1850 notes
MEDICATIONS ON ADMISSION: — 1830 notes
Cranial Nerves: — 1528 notes
HOSPITAL COURSE: — 1411 notes
FINAL DIAGNOSIS: — 837 notes
CARE RECOMMENDATIONS: — 800 notes
DISCHARGE INSTRUCTIONS: — 751 notes
PAST SURGICAL HISTORY: 

### Filter reached stopping phrases notes

In [ ]:
# import pandas as pd
# def contains_stop_phrase(text, stop_phrases):
#     return any(phrase in text for phrase in stop_phrases)

# # synthesize_path = '/content/train.parquet'
# synthesize_path = '/content/synthesize.parquet'
# synthesize_df = pd.read_parquet(synthesize_path)

# # Filter the rows where the text ends with one of the stop phrases
# matched_rows = synthesize_df[synthesize_df['text'].apply(lambda text: contains_stop_phrase(text, stop_phrases))]
# print("Number of rows that end with a stop phrase:", len(matched_rows))


# matched_rows = matched_rows.reset_index()
# print(matched_rows)
# matched_rows.head()
# matched_rows.to_parquet("ended_synthesize.parquet", index=False)

Number of rows that end with a stop phrase: 865


### BioGPT

In [ ]:
# import kagglehub

# kagglehub.notebook_output_download(
#     "chanhainguyen/mimic-biogpt"  # replace with your notebook's path
# )

Extracting files...


'/root/.cache/kagglehub/notebooks/chanhainguyen/mimic-biogpt/output/versions/1'

In [ ]:
# !mkdir train_output
# !mv /root/.cache/kagglehub/notebooks/chanhainguyen/mimic-biogpt/output/versions/1 /content/train_output

mkdir: cannot create directory ‘train_output’: File exists


In [ ]:
# from transformers import pipeline
# checkpoint = 2
# generator = pipeline("text-generation", model=f"/content/train_output/1/output/{checkpoint}")
# prompt = "Admission Date:  [**2196-10-7**]              Discharge Date:   [**2196-10-13**]\n\nDate of Birth:  [**2168-11-16**]             Sex:   F\n\nService: "

Device set to use cuda:0


In [ ]:
# output = generator(prompt, max_new_tokens=800)
# # print the output
# print(output[0]["generated_text"])

Admission Date:  [**2196-10-7**]              Discharge Date:   [**2196-10-13**]

Date of Birth:  [**2168-11-16**]             Sex:   F

Service:  Endocrine -

ADDENDUM:  This is a significant change for the
female pelvic ultrasound site and the location of the
posterior pelvic ultrasound site.  The location of the cervical
tube site is unchanged.  The cervical and cervical
tube site is unchanged for any other reasons.

IMPRESSION:  The cervical and cervical
tube sites are unchanged.  The cervical and cervical
tube site are unchanged.  This is concerning for
the pelvic ultrasound site and the location of the pelvic
tube site.  There is no evidence of cervical or cervical/cervical
surgery.  The cervical and cervical pelvic
tube sites are unchanged.  The cervical and cervical
tube site are unchanged.  The cervical and cervical pelvic and cervical
tube sites are unchanged.  The cervical and cervical pelvic and pelvic
tube sites are unchanged.  The cervical and cervical pelvic and pelvic
t

In [ ]:
# output = generator(
#     prompt,
#     max_new_tokens=800,
#     # repetition_penalty=1.2,  # 🚫 phạt lặp lại
#     pad_token_id=tokenizer.eos_token_id
# )

# # print the output
# print(output[0]["generated_text"])

Admission Date:  [**2196-10-7**]              Discharge Date:   [**2196-10-13**]

Date of Birth:  [**2168-11-16**]             Sex:   F

Service:  SURGERY

Allergies:
Penicillins /

Attending:[**First Name3 (LF) 2596**]
Chief Complaint:
Acute right foot pain.

Major Surgical or Invasive Procedure:
Right foot flexion/extremity bending:
[**2196-10-11**] 12:43PM

History of Present Illness:
Patient was admitted to the [**Hospital1 18**] [**Location (un) **] [**Location (un) **] for a
headache. He was found to have a right foot pain and
extremity bending with a right foot compression. Patient was transferred to
the floor for further management.

On arrival to the floor, he had a headache, which was
displaced.  By the time of arrival, patient's
pulmonary distress was not controlled.  Patient was
complicated by chest pain and chest pain.  He was
stable and extubated.  He was extubated and
extubated in good condition.  At that time, he received a
CXR.  However, his pain was not controlled.  H

In [ ]:
# from transformers import AutoModelForCausalLM
# from transformers import AutoTokenizer

# model = AutoModelForCausalLM.from_pretrained(f"/content/train_output/1/output/{checkpoint}")
# tokenizer = AutoTokenizer.from_pretrained(f"/content/train_output/1/output/{checkpoint}")
# tokenized = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)
# inputs = tokenized["input_ids"]
# attention_mask = tokenized["attention_mask"]

In [ ]:

# outputs = model.generate(
#     inputs,
#     # attention_mask=attention_mask,
#     max_new_tokens=800,
#     do_sample=True,
#     # top_k=50,
#     # top_p=0.95,
#     temperature=0.7,
#     pad_token_id=tokenizer.eos_token_id # Set pad_token_id
# )

# output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
# print(output[0])

Admission Date:  [**2196-10-7**]              Discharge Date:   [**2196-10-13**]

Date of Birth:  [**2168-11-16**]             Sex:   F

Service:  MEDICINE

Allergies:
Patient recorded as having No Known Allergies to Drugs

Attending:[**First Name3 (LF) **]
Chief Complaint:
Percussion

Major Surgical or Invasive Procedure:
1.
2.
3.
4.
5.
6.
7.
8.
9.
10.

Discharge Medications:
1. Fluticasone 50 mg Tablet Sig: 1-2 Tablets PO Q4H (every 4
hours) as needed for pain.
2. Lipitor 30 mg Tablet Sig: 1-2 Tablets PO Q4H (every 4
hours) as needed for pain.
3. Levofloxacin 2 mg Tablet Sig: 1-2 Tablets PO QHS (every 4
hours) as needed for pain.
4. Senna 8.5 mg Tablet Sig: One (1) Tablet PO Q4H (every 4
hours) as needed for pain.
5. Metronidazole 50 mg Tablet Sig: 1-2 Tablets PO Q6H (every 6
hours) as needed for pain.
6. Potassium Chloride 2.5 mg/5 mL Suspension Sig: One (1)
Tablet PO TID (3 times a day).
7. Insulin Glargine 200 mg/mL Suspension Sig: One (1) Appl Topical
(3 times a day).
8. Metronid

### BioGPT Lare

In [ ]:
# laod model from hugging face ChanBeDu/biogpt-note-synthesis
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_name = "ChanBeDu/BioGPT-sec-synthesis-b40"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
generator = pipeline("text-generation", model=model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/982 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

ImportError: You need to install sacremoses to use BioGptTokenizer. See https://pypi.org/project/sacremoses/ for installation.

### Long note generating

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def generate_long_note(
    generate_id: int,
    prompt: str,
    codes: list[str],
    model,
    tokenizer,
    chunk_size: int = 512,
    average_token_per_code=331,
    stop_phrases: list = None,
    do_sample: bool = True,
    temperature: float = 0.7,
    top_p: float = 0.9,
) -> str:
    # 👉 Move model to CUDA
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Compute max token budget
    max_total_tokens = average_token_per_code * len(codes)
    model_max_len = model.config.max_position_embeddings  # Typically 2048

    # print(f"Max total token: {max_total_tokens}")
    # 👉 Encode prompt and move to CUDA
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    full_generated = input_ids.clone()

    turn_count = 0

    reach_end_phrase = False
    while full_generated.shape[-1] < max_total_tokens:
        # print(f"Generating turn: {turn_count}")

        # Build context window
        context_len = min(model_max_len - chunk_size, full_generated.shape[-1])
        context_window = full_generated[:, -context_len:].to(device)

        # 👉 Generate with model on CUDA
        output = model.generate(
            input_ids=context_window,
            max_new_tokens=chunk_size,
            do_sample=do_sample,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

        # Get new tokens
        new_tokens = output[:, context_window.shape[-1]:].to(device)
        if new_tokens.shape[-1] == 0:
            break

        full_generated = torch.cat([full_generated, new_tokens], dim=-1)

        # Optional early stopping
        generated_text = tokenizer.decode(full_generated[0], skip_special_tokens=True)
        if stop_phrases and any(phrase in generated_text for phrase in stop_phrases):
            reach_end_phrase = True
            break

        turn_count += 1

    if not reach_end_phrase:
        print("Generating end before reaching stop phrase, generate id ", generate_id)

    return tokenizer.decode(full_generated[0], skip_special_tokens=True), full_generated[0]


In [ ]:
import pandas as pd
from datasets import Dataset
from typing import List, Dict

diagnosis_code_des_path = "/content/src/diagnosis_code_des.csv"
procedure_code_des_path = "/content/src/procedure_code_des.csv"
diagnosis_code_des = pd.read_csv(diagnosis_code_des_path)
procedure_code_des = pd.read_csv(procedure_code_des_path)

def build_code_description_map(code_df: pd.DataFrame) -> Dict[str, str]:
    return {str(row['code']): row['des'] for _, row in code_df.iterrows()}

diagnosis_map = build_code_description_map(diagnosis_code_des) # run code in read code description under Load MIMIC section
procedure_map = build_code_description_map(procedure_code_des) # run code in read code description under Load MIMIC section

In [ ]:
def format_icd_context(diagnosis_codes: List[str], procedure_codes: List[str],
                       diagnosis_map: Dict[str, str], procedure_map: Dict[str, str]) -> str:
    def describe(codes, mapping):
        return "; ".join(
            f"{code} ({mapping.get(code, 'unknown')})"
            for code in codes
        ) if codes else "None"

    diag_str = describe(diagnosis_codes, diagnosis_map)
    proc_str = describe(procedure_codes, procedure_map)

    return f"Diagnosis Codes: {diag_str}\nProcedure Codes: {proc_str}\n\n"

def get_prompt_and_codes(row):
  prompt = format_icd_context(
      diagnosis_codes=row["diagnosis_codes"],
      procedure_codes=row["procedure_codes"],
      diagnosis_map=diagnosis_map,
      procedure_map=procedure_map
  )
  codes = row["diagnosis_codes"] + row["procedure_codes"]
  return prompt, codes

def get_prompt_and_codes_from_augmented_codes(diagnosis_codes, procedure_codes):
  prompt = format_icd_context(
      diagnosis_codes=list(diagnosis_codes),
      procedure_codes=list(procedure_codes),
      diagnosis_map=diagnosis_map,
      procedure_map=procedure_map
  )
  return prompt, augmented_set

#### Testing generate

##### Parsing sessions

In [ ]:
import re
from typing import Dict, List

def extract_note_sections(full_text: str, known_headings: List[str], is_real=False) -> Dict[str, str]:
    # 1. Cut off everything before and including 'Procedure Codes: None'
    note_text = full_text
    if not is_real:
      split_marker = re.search(r'procedure codes:\s*none', full_text, re.IGNORECASE)
      if not split_marker:
          raise ValueError("Could not find 'Procedure Codes: None' marker.")
      note_text = full_text[split_marker.end():].strip()

    # 2. Normalize and prepare headings
    heading_patterns = [
        re.escape(h.strip()) for h in known_headings
    ]
    heading_regex = r'(' + '|'.join(heading_patterns) + r')\s*:?'  # support colon and no colon

    # 3. Find all heading matches
    matches = list(re.finditer(heading_regex, note_text, re.IGNORECASE))

    # 4. Extract sections
    sections = {}
    for i, match in enumerate(matches):
        heading = match.group(1).strip()
        start = match.end()
        end = matches[i+1].start() if i+1 < len(matches) else len(note_text)
        content = note_text[start:end].strip()
        sections[heading] = content


    for heading, content in sections.items():
        print(f"== {heading} ==\n{content}\n")
    return sections

##### Load Real-releval note dataset

In [ ]:
from datasets import load_dataset

# Load the dataset
augmented_note_id = load_dataset("ChanBeDu/augmented_icd9_note_id")

README.md:   0%|          | 0.00/864 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/865 [00:00<?, ? examples/s]

In [ ]:
# Extract note_ids from dataset_a
note_ids_to_keep = set([ids[0] for ids in augmented_note_id['train']["match_note_id"] if ids] )

# Filter dataset_b to keep only rows with note_id in dataset_a
real_relevant_dataset = mimic['train'].filter(lambda x: x["note_id"] in note_ids_to_keep)

# Check result
print(len(real_relevant_dataset))

755


In [ ]:
bid = 3
print(real_relevant_dataset['diagnosis_codes'][bid])
print(real_relevant_dataset['procedure_codes'][bid])

# for i in range(len(real_relevant_dataset)):
#   if len(real_relevant_dataset['procedure_codes'][i]):
#     print(i)
#     break

['576.2', '311', '300.00', '338.29', 'V45.4', '305.1', '518.0']
['50.3']


In [ ]:
sections = extract_note_sections(real_relevant_dataset['text'][bid], common_headings, True)

== Chief Complaint: ==
Right hepatic duct stricture.

== Major Surgical or Invasive Procedure: ==
[**2161-12-25**] right lobectomy for common hepatic stricture

== History of Present Illness: ==
42M with remote h/o lap chole previously p/w [**Month/Day/Year 5283**] pain, s/p ERCP
[**11-18**] showing R hepatic biliary stricture, thougt to be
postsurgical; s/p PTC external biliary drain into R anterior
biliary duct [**11-19**], PTC repositioned [**11-30**]. Bile cultures grew
out sparse Lactobacillus. Discharged from hospital [**12-3**] on 21 d
course of Augmentin. Returned [**12-10**] with diarrhea, [**Month/Year (2) 5283**] abd pain,
nausea, po intolerance with 7 pound wt loss

== Past Medical History: ==
Bile duct stricture
depression and anxiety.
ERCP [**2161-11-18**] showing R hepatic biliary stricture
PTC external biliary drain into R anterior biliary duct
[**2161-11-19**], PTC repositioned [**2161-11-30**]
chronic back pain s/p fall down stairs 1 yr prior

PSH: lap chole [**2145**

In [ ]:
real_relevant_dataset['text'][bid]

'Admission Date:  [**2161-12-22**]              Discharge Date:   [**2161-12-31**]\n\nDate of Birth:  [**2119-9-5**]             Sex:   M\n\nService: SURGERY\n\nAllergies:\nPatient recorded as having No Known Allergies to Drugs\n\nAttending:[**First Name3 (LF) 668**]\nChief Complaint:\nRight hepatic duct stricture.\n\n\nMajor Surgical or Invasive Procedure:\n[**2161-12-25**] right lobectomy for common hepatic stricture\n\n\nHistory of Present Illness:\n42M with remote h/o lap chole previously p/w [**Month/Day/Year 5283**] pain, s/p ERCP\n[**11-18**] showing R hepatic biliary stricture, thougt to be\npostsurgical; s/p PTC external biliary drain into R anterior\nbiliary duct [**11-19**], PTC repositioned [**11-30**]. Bile cultures grew\nout sparse Lactobacillus. Discharged from hospital [**12-3**] on 21 d\ncourse of Augmentin. Returned [**12-10**] with diarrhea, [**Month/Year (2) 5283**] abd pain,\nnausea, po intolerance with 7 pound wt loss\n\nPast Medical History:\nBile duct stricture\

##### Build test prompt

In [ ]:
# # row = mimic["train"][0]
# # prompt, codes = get_prompt_and_codes(row)

# augmented_set = augmented_code_sets[0]
# prompt, codes = get_prompt_and_codes_from_augmented_dx_codes(augmented_set)
# print(prompt)
# print(codes)

In [ ]:
augmented_set = real_relevant_dataset['diagnosis_codes'][bid]
prompt, codes = get_prompt_and_codes_from_augmented_codes(augmented_set,[])
print(prompt)
print(codes)

Diagnosis Codes: 576.2 ( Obstruction of bile duct); 311 ( Depressive disorder, not elsewhere classified); 300.00 ( Anxiety state, unspecified); 338.29 ( Other chronic pain); V45.4 ( Arthrodesis status); 305.1 ( Tobacco use disorder); 518.0 ( Pulmonary collapse)
Procedure Codes: None


['576.2', '311', '300.00', '338.29', 'V45.4', '305.1', '518.0']


##### Long note Generate

In [ ]:
generated_note, output_ids = generate_long_note(
    0,
    prompt=prompt,
    codes=codes,
    model=model,
    tokenizer=tokenizer,
    chunk_size=1024,
    stop_phrases=common_endings,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

print(generated_note)

Generating end before reaching stop phrase, generate id  0
Diagnosis Codes: 576.2 (Obstruction of bile duct); 311 (Depressive disorder, not elsewhere classified); 300.00 (Anxiety state, unspecified); 338.29 (Other chronic pain); V45.4 (Arthrodesis status); 305.1 (Tobacco use disorder); 518.0 (Pulmonary collapse) Procedure Codes: None Admission Date: [* * 2190-11-25 * *] Discharge Date: [* * 2190-11-28 * *] Date of Birth: [* * 2120-9-14 * *] Sex: M Service: MEDICINE Allergies: Patient recorded as having No Known Allergies to Drugs Attending: [* * First Name3 (LF) 1835 * *] Chief Complaint: Abdominal pain, nausea, vomiting Major Surgical or Invasive Procedure: None History of Present Illness: 46 year old man with past medical history significant for depression, anxiety, substance abuse who presented to the ED for abdominal pain, nausea, vomiting. He reports that he had been feeling nauseous, but that the nausea and vomiting started a couple of days ago. He reports that he had been unable

In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
sections = extract_note_sections(generated_note, common_headings)

== Chief Complaint: ==
Abdominal pain, nausea, vomiting

== Major Surgical or Invasive Procedure: ==
None

== History of Present Illness: ==
46 year old man with past medical history significant for depression, anxiety, substance abuse who presented to the ED for abdominal pain, nausea, vomiting. He reports that he had been feeling nauseous, but that the nausea and vomiting started a couple of days ago. He reports that he had been unable to take his medications because he was "too anxious" and was unable to take his medications due to the nausea and vomiting. He was seen by his PCP, [* * Name10 (NameIs) * *], who recommended an outpatient gastroenterology consult. He reports that he was recently hospitalized for right lower quadrant pain, and was found to have an obstructing stone in the CBD and a dilated CBD. He was admitted to the MICU for monitoring, and was transferred to the medicine floor on [* * 11-25 * *].. In the ED, initial VS were T 97.6, BP 130 / 60, HR 100, RR 20, O2 sat 9

##### Friendly testing

In [ ]:
# def get_prompt_token_length(prompt, tokenizer):
#   prompt_tokenized = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)
#   return len(prompt_tokenized["input_ids"][0])

# def get_last_output_prompt(output, tokenizer, length=1024):
#   last_output = output[0]["generated_text"]
#   last_output_tokenized = tokenizer(last_output, return_tensors="pt", return_attention_mask=True)
#   # decode last 1024 token of the last ouput to be new input
#   last_output_prompt = tokenizer.decode(last_output_tokenized["input_ids"][0][-length:], skip_special_tokens=True)
#   return last_output_prompt, length

In [ ]:
# prompt = "Diagnosis Codes: 585.6 (End stage renal disease); 038.9 (Unspecified septicemia); 427.31 (Atrial fibrillation); 078.5 (Cytomegaloviral disease); 242.90 (Thyrotoxicosis without mention of goiter or other cause, and without mention of thyrotoxic crisis or storm); 710.0 (Systemic lupus erythematosus); 453.82 (Acute venous embolism and thrombosis of deep veins of upper extremity) Procedure Codes: None"
# output = generator(prompt, max_new_tokens=2048-get_prompt_token_length(prompt, tokenizer))
# print(output[0]["generated_text"])

In [ ]:
# last_output_prompt, length = get_last_output_prompt(output, tokenizer, length=1024)
# output2 = generator(last_output_prompt, max_new_tokens=length)
# print(output2[0]["generated_text"])

In [ ]:
# last_output_prompt, length = get_last_output_prompt(output2, tokenizer, length=1024)
# output3 = generator(last_output_prompt, max_new_tokens=length)
# print(output3[0]["generated_text"])

In [ ]:
# prompt = "Diagnosis Codes: 576.2 (Obstruction of bile duct); 311 (Depressive disorder, not elsewhere classified); 300.00 (Anxiety state, unspecified); 338.29 (Other chronic pain); V45.4 (Arthrodesis status); 305.1 (Tobacco use disorder); 518.0 (Pulmonary collapse) Procedure Codes: None Admission Date:  [**2161-12-22**]              Discharge Date:   [**2161-12-31**] Date of Birth:  [**2119-9-5**]             Sex:   M Service: SURGERY Allergies: Patient recorded as having No Known Allergies to Drugs Attending:[**First Name3 (LF) 668**] Chief Complaint: Right hepatic duct stricture. Major Surgical or Invasive Procedure: [**2161-12-25**] right lobectomy for common hepatic stricture History of Present Illness: 42M with remote h/o lap chole previously p/w [**Month/Day/Year 5283**] pain, s/p ERCP [**11-18**] showing R hepatic biliary stricture, thougt to be postsurgical; s/p PTC external biliary drain into R anterior biliary duct [**11-19**], PTC repositioned [**11-30**]. Bile cultures grew out sparse Lactobacillus. Discharged from hospital [**12-3**] on 21 d course of Augmentin. Returned [**12-10**] with diarrhea, [**Month/Year (2) 5283**] abd pain, nausea, po intolerance with 7 pound wt loss Past Medical History:"
prompt = "History of Present Illness: 42M with remote h/o lap chole previously p/w [**Month/Day/Year 5283**] pain, s/p ERCP [**11-18**] showing R hepatic biliary stricture, thougt to be postsurgical; s/p PTC external biliary drain into R anterior biliary duct [**11-19**], PTC repositioned [**11-30**]. Bile cultures grew out sparse Lactobacillus. Discharged from hospital [**12-3**] on 21 d course of Augmentin. Returned [**12-10**] with diarrhea, [**Month/Year (2) 5283**] abd pain, nausea, po intolerance with 7 pound wt loss Past Medical History: Anxiety / Depression h / o lap chole  Social History: Lives with: wife, previously worked as a consultant in health services; has occasional [* * Name (NI) 5283 * *] tobacco use; denies any alcohol use; denies other drug use Family History:"
output = generator(prompt, max_new_tokens=100)
print(output[0]["generated_text"])

History of Present Illness: 42M with remote h/o lap chole previously p/w [**Month/Day/Year 5283**] pain, s/p ERCP [**11-18**] showing R hepatic biliary stricture, thougt to be postsurgical; s/p PTC external biliary drain into R anterior biliary duct [**11-19**], PTC repositioned [**11-30**]. Bile cultures grew out sparse Lactobacillus. Discharged from hospital [**12-3**] on 21 d course of Augmentin. Returned [**12-10**] with diarrhea, [**Month/Year (2) 5283**] abd pain, nausea, po intolerance with 7 pound wt loss Past Medical History: Anxiety / Depression h / o lap chole  Social History: Lives with: wife, previously worked as a consultant in health services; has occasional [* * Name (NI) 5283 * *] tobacco use; denies any alcohol use; denies other drug use Family History: N / C Physical Exam: ADMISSION EXAM: Vitals: 98.2 97 / 55 153 / 105 18 98% RA General: Alert, oriented, no acute distress HEENT: Sclera anicteric, MMM, oropharynx clear, EOMI, PERRL Neck: supple, JVP not elevated, no L

In [ ]:
prompt = "Diagnosis Codes: 576.2 (Obstruction of bile duct); 311 (Depressive disorder, not elsewhere classified); 300.00 (Anxiety state, unspecified); 338.29 (Other chronic pain); V45.4 (Arthrodesis status); 305.1 (Tobacco use disorder); 518.0 (Pulmonary collapse) Procedure Codes: None Admission Date:  [**2161-12-22**]              Discharge Date:   [**2161-12-31**] Date of Birth:  [**2119-9-5**]             Sex:   M Service: SURGERY Allergies: Patient recorded as having No Known Allergies to Drugs Attending:[**First Name3 (LF) 668**] Chief Complaint: Right hepatic duct stricture. Major Surgical or Invasive Procedure: [**2161-12-25**] right lobectomy for common hepatic stricture History of Present Illness: 42M with remote h/o lap chole previously p/w [**Month/Day/Year 5283**] pain, s/p ERCP [**11-18**] showing R hepatic biliary stricture, thougt to be postsurgical; s/p PTC external biliary drain into R anterior biliary duct [**11-19**], PTC repositioned [**11-30**]. Bile cultures grew out sparse Lactobacillus. Discharged from hospital [**12-3**] on 21 d course of Augmentin. Returned [**12-10**] with diarrhea, [**Month/Year (2) 5283**] abd pain, nausea, po intolerance with 7 pound wt loss Past Medical History:"
tokenize = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)
output = model.generate(
    input_ids=tokenize["input_ids"].to(model.device),
    attention_mask=tokenize["attention_mask"].to(model.device),
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
ouput_text = tokenizer.batch_decode(output, skip_special_tokens=True)
print(ouput_text[0])

Diagnosis Codes: 576.2 (Obstruction of bile duct); 311 (Depressive disorder, not elsewhere classified); 300.00 (Anxiety state, unspecified); 338.29 (Other chronic pain); V45.4 (Arthrodesis status); 305.1 (Tobacco use disorder); 518.0 (Pulmonary collapse) Procedure Codes: None Admission Date: [* * 2161-12-22 * *] Discharge Date: [* * 2161-12-31 * *] Date of Birth: [* * 2119-9-5 * *] Sex: M Service: SURGERY Allergies: Patient recorded as having No Known Allergies to Drugs Attending: [* * First Name3 (LF) 668 * *] Chief Complaint: Right hepatic duct stricture. Major Surgical or Invasive Procedure: [* * 2161-12-25 * *] right lobectomy for common hepatic stricture History of Present Illness: 42M with remote h / o lap chole previously p / w [* * Month / Day / Year 5283 * *] pain, s / p ERCP [* * 11-18 * *] showing R hepatic biliary stricture, thougt to be postsurgical; s / p PTC external biliary drain into R anterior biliary duct [* * 11-19 * *], PTC repositioned [* * 11-30 * *]. Bile cultur

##### Section Knowledge Generate

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load both datasets
match_note_id = load_dataset("ChanBeDu/augmented_icd9_note_id_full")["train"]
match_section_length = load_dataset("ChanBeDu/match_mimic_section_length")["train"]

In [ ]:
# Convert to pandas
df_match_note_id = match_note_id.to_pandas()
df_section_length = match_section_length.to_pandas()

# Optional: rename duplicate columns (like 'note_id')
df_match_note_id = df_match_note_id.rename(columns={'note_id': 'note_id_match_note_id'})
df_section_length = df_section_length.rename(columns={'note_id': 'note_id_section'})

# Merge on match_note_id <-> note_id_section
merged_df = pd.merge(
    df_match_note_id,
    df_section_length,
    left_on='match_note_id',
    right_on='note_id_section',
    how='inner'  # or 'left', 'outer' depending on your needs
)

# Convert back to Hugging Face dataset if needed
from datasets import Dataset
merged_dataset = Dataset.from_pandas(merged_df)

#### Cool thing happen: synthesize the whole augmeted sets

In [ ]:
from google.colab import files

def download_result(start, end):
  # Replace this with your actual file path
  file_path = f"/content/result{start}_{end}_df.parquet"

  # Download the file
  files.download(file_path)

In [ ]:
import pandas as pd
from tqdm import tqdm

result_df = pd.DataFrame(columns=["id", "rare codes", "prompt", "augmented codes", "text"])

# start = 0
# end = 40
# start = 40
# end = 80
# start = 80
# end = 120
# start = 120
# end = 160
# start = 160
# end = 200
# start = 200
# end = 240
# start = 240
# end = 280
# start = 280
# end = 320

# start = 1360
# end = 1380

# start = 1600
# end = 1800

# start = 2060
# end = 2100

# start = 2390
# end = 2400

# start = 2580
# end = 2600

# start = 2630
# end = 2640


# start = 2660
# end = 2680


# start = 2680
# end = 2720

start = 2790
end = 2800


# start = 120
# end = 200
i = 0
for augmented_set in tqdm(augmented_code_sets[start:end]):

  prompt, codes = get_prompt_and_codes_from_augmented_dx_codes(augmented_set)
  rare_codes = [code for code in codes if code in rare_codes]

  generated_note, output_ids = generate_long_note(
    generate_id=start+i,
    prompt=prompt,
    codes=codes,
    model=model,
    tokenizer=tokenizer,
    chunk_size=1024,
    stop_phrases=stop_phrases,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
  )

  result_df = result_df._append({
    "id": i,
    "rare codes": rare_codes,
    "prompt": prompt,
    "augmented codes": augmented_set,
    "text": generated_note
  }, ignore_index=True)

  if i > 0 and i % 10 == 0:
    result_df.to_parquet(f"result{start}_{start + i}_df.parquet")
    download_result(start, start + i)
  i += 1

 10%|█         | 1/10 [08:52<1:19:51, 532.38s/it]

Generating end before reaching stop phrase, generate id  2790


 20%|██        | 2/10 [12:26<46:02, 345.34s/it]  

Generating end before reaching stop phrase, generate id  2791


 30%|███       | 3/10 [17:56<39:28, 338.41s/it]

...

Generating end before reaching stop phrase, generate id  2796


 90%|█████████ | 9/10 [41:18<03:36, 216.56s/it]

Generating end before reaching stop phrase, generate id  2798


100%|██████████| 10/10 [49:41<00:00, 298.18s/it]

Generating end before reaching stop phrase, generate id  2799


In [ ]:
#save result to parquet
result_df.to_parquet(f"result{start}_{end}_df.parquet")

In [ ]:
result_df.head()

,id,rare codes,prompt,augmented codes,text
0,0,[V49.72],"Diagnosis Codes: 285.9 ( Anemia, unspecified);...","[285.9, 414.8, 785.51, 427.89, 276.1, 790.29, ...","Diagnosis Codes: 285.9 (Anemia, unspecified); ..."
1,1,[V49.72],Diagnosis Codes: 996.49 ( Other mechanical com...,"[996.49, 564.00, 272.4, V49.72, 733.00, 285.1,...",Diagnosis Codes: 996.49 (Other mechanical comp...
2,2,[V49.72],Diagnosis Codes: V49.72 ( Other toe(s) amputat...,"[V49.72, 427.32, 427.31, 365.9, 427.5, 440.4, ...",Diagnosis Codes: V49.72 (Other toe (s) amputat...
3,3,[V49.72],Diagnosis Codes: 852.01 ( Subarachnoid hemorrh...,"[852.01, 272.4, 443.9, 802.4, E849.9, 403.90, ...",Diagnosis Codes: 852.01 (Subarachnoid hemorrha...
4,4,[V49.72],Diagnosis Codes: 403.91 ( Hypertensive chronic...,"[403.91, V49.72, V43.65, 458.29, 530.81, V45.1...",Diagnosis Codes: 403.91 (Hypertensive chronic ...


In [ ]:
download_result(start, end)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Constraint Generation

In [ ]:
# from transformers import BartForConditionalGeneration, BartTokenizer, PhrasalConstraint

# model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
# tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# # encode input
# inputs = tokenizer(input_text, return_tensors="pt")

# # define constraints
# constraints = [
#     PhrasalConstraint(tokenizer.encode("hyperglycemia", add_special_tokens=False)),
#     PhrasalConstraint(tokenizer.encode("hypertension", add_special_tokens=False))
# ]

# # generate
# outputs = model.generate(
#     **inputs,
#     constraints=constraints,
#     num_beams=5,
#     max_length=128
# )
